In [1]:
import pandas as pd
from pathlib import Path

In [2]:
# set the file path and the sheet names to explore
# base = Path(r"d:\yzy\CMU\26SPRING\95451 pm\project\VSP Vision Datasets")
import os 
base = Path(os.getcwd()+"/VSP Vision Datasets")
sources = [
    ("AO (demand)", "AO-BI275 DEMAND KC KP LA LS KO KS 12.17.25.xlsx", ["Brand View"]),
    ("Calvin Klein", "Calvin Klein_Sept24 ATP.xlsx", ["SUN", "OPH"]),
    ("Lacoste", "LACOSTE_Sept24 ATP.xlsx", ["LACOSTE OPTICAL", "LACOSTE SUN"]),
    ("Nike", "Nike_Sept24 ATP.xlsm", ["Nike Sept 24 Optical", "Nike Sept 24 Sun"]),
]

## 1. Preprocess 'demand' Data

In [3]:
# load and clean the 'damand' data
df_ao_raw = pd.read_excel(
    base / "AO-BI275 DEMAND KC KP LA LS KO KS 12.17.25.xlsx",
    sheet_name="Brand View",
    header=None,
)

n_meta = 7
meta_names = ["Collection", "BrandLine", "Material", "StyleCode", "GridValue", "Style", "Region"]

month_cols = df_ao_raw.iloc[1, n_meta:].tolist()   
demand = df_ao_raw.iloc[2:].copy()                  
demand.columns = meta_names + month_cols

demand = demand[demand.iloc[:, 0].astype(str).str.upper() != "COLLECTION"].copy()

for i in range(n_meta, demand.shape[1]):
    demand.iloc[:, i] = pd.to_numeric(demand.iloc[:, i], errors="coerce")


def _parse_style(val):
    if pd.isna(val):
        return pd.Series({"Size": "", "Color": ""})
    
    txt = str(val).strip()
    parts = txt.split("/")
    n = len(parts)
    
    if n < 2:
        return pd.Series({"Size": "", "Color": ""})
    
    size = ""
    color_start_idx = 1
    
    for i in range(1, n):
        item = parts[i].strip()
        if item.isdigit():
            size = item
            color_start_idx = i + 1
        else:
            break
            
    if not size and n >= 2:
        size = "" 
        color_start_idx = 1 if n == 2 else 2 

    color = "/".join(parts[color_start_idx:]).strip()
    
    if not color and n > color_start_idx - 1:
        pass

    return pd.Series({"Size": size, "Color": color})

style_extras = demand["Style"].apply(_parse_style)
demand = demand.drop(columns=["Size", "Color"], errors='ignore')
demand = pd.concat([demand, style_extras], axis=1)

print(demand.head())

  Collection         BrandLine Material StyleCode GridValue  \
2         KC  CALVIN KLEIN SUN    45073  CK20541S   5719001   
3         KC  CALVIN KLEIN SUN    45073  CK20541S   5719001   
4         KC  CALVIN KLEIN SUN    45073  CK20541S   5719235   
5         KC  CALVIN KLEIN SUN    45073  CK20541S   5719235   
6         KC  CALVIN KLEIN SUN    45073  CK20541S   5719605   

                  Style Region 09/2023 10/2023 11/2023  ... 02/2024 03/2024  \
2     CK20541S/57/BLACK   AMER    26.0    28.0    27.0  ...    24.0    21.0   
3     CK20541S/57/BLACK   EMEA    10.0    18.0    11.0  ...    36.0    84.0   
4  CK20541S/57/DARK TOR   AMER    33.0    39.0    14.0  ...    15.0    16.0   
5  CK20541S/57/DARK TOR   EMEA    21.0    15.0    11.0  ...    61.0    55.0   
6   CK20541S/57/CRYSTAL   AMER    11.0    21.0    15.0  ...     8.0    11.0   

  04/2024 05/2024 06/2024 07/2024 08/2024 Overall Result Size     Color  
2    35.0    21.0    20.0    32.0    16.0          285.0   57     BLACK 

In [4]:
print('demand shape:', demand.shape)

demand shape: (1866, 22)


In [5]:
# first drop rows without a style (they're empty/aggregate rows)
dropped = demand['Style'].isna().sum()
print(f"dropping {dropped} rows with missing Style")
demand = demand[demand['Style'].notna()].copy()

dropping 402 rows with missing Style


In [6]:
# reshape the demand data to long format
# first drop any styles that were missing (already done earlier)
demand_long = demand.melt(
    id_vars=meta_names + ["Size", "Color"],
    value_vars=month_cols,
    var_name="Month",
    value_name="Demand",
)

# drop any non-month labels such as "Overall Result" before converting
mask_valid = demand_long["Month"].astype(str).str.match(r"^\d{2}/\d{4}$")
demand_long = demand_long[mask_valid].copy()

# diagnostic prints
print("month_cols:", month_cols)
print("filtered demand_long shape:", demand_long.shape)
print(demand_long.head())

# align the date format to "YYYY-MM" using the correct pattern for MM/YYYY
# (the original strings are like '09/2023')
demand_long["Month"] = pd.to_datetime(
    demand_long["Month"],
    format="%m/%Y",         
    errors="coerce"
).dt.strftime("%Y-%m")

# check NaNs in the melted dataframe
nan_counts = demand_long.isna().sum()
print("NaN counts in demand_long:\n", nan_counts)


month_cols: ['09/2023', '10/2023', '11/2023', '12/2023', '01/2024', '02/2024', '03/2024', '04/2024', '05/2024', '06/2024', '07/2024', '08/2024', 'Overall Result']
filtered demand_long shape: (17568, 11)
  Collection         BrandLine Material StyleCode GridValue  \
0         KC  CALVIN KLEIN SUN    45073  CK20541S   5719001   
1         KC  CALVIN KLEIN SUN    45073  CK20541S   5719001   
2         KC  CALVIN KLEIN SUN    45073  CK20541S   5719235   
3         KC  CALVIN KLEIN SUN    45073  CK20541S   5719235   
4         KC  CALVIN KLEIN SUN    45073  CK20541S   5719605   

                  Style Region Size     Color    Month Demand  
0     CK20541S/57/BLACK   AMER   57     BLACK  09/2023   26.0  
1     CK20541S/57/BLACK   EMEA   57     BLACK  09/2023   10.0  
2  CK20541S/57/DARK TOR   AMER   57  DARK TOR  09/2023   33.0  
3  CK20541S/57/DARK TOR   EMEA   57  DARK TOR  09/2023   21.0  
4   CK20541S/57/CRYSTAL   AMER   57   CRYSTAL  09/2023   11.0  
NaN counts in demand_long:
 Collec

In [7]:
# merge by style region and month/ deduplicate
# This will automatically fill the demand=nan with 0.0
demand_monthly = demand_long.groupby(meta_names + ["Size", "Color",'Month'], as_index=False)["Demand"].sum()

In [8]:
# count rows where demand is exactly zero
demand_zero = (demand_monthly['Demand'] == 0).sum()
print(f"rows in demand_monthly with Demand == 0: {demand_zero}")

rows in demand_monthly with Demand == 0: 2563


### Map Color

In [9]:
demand_monthly["Color"].unique()

array(['BLACK', 'DARK TOR', 'CRYSTAL', 'BROWN HA', 'AVIO', 'ROSE',
       'BROWN', 'SAND', 'BLUE', 'OYSTER', 'BUTTERSC', 'AZURE', 'GOLD',
       'DARK HAV', 'VIOLET', 'GREY', 'TOKYO HA', 'MATTE BL', 'SILVER',
       'MATTE GO', 'PETROL', 'BURGUNDY', 'GREY/BEI', 'MINT', 'VIOLET/B',
       'BLACK/PI', 'STRIPED', 'GREEN/MI', 'CHERRY/R', 'CHALK', 'PEACH',
       'TAUPE', 'BLACK/W', 'SATIN B', 'BLACK/V', 'OBSIDIA', 'MATTE',
       'DARK', 'ANTHR', 'MATTE B', 'MT CRYS', 'MATTE D', 'MATTE N',
       'BLACK-F', 'BLACK-P', 'ANTHRAC', 'MIDNIGH', 'BLACK/C', 'SATIN A',
       'MATTE W', 'GRIDIRO', 'DARK GR', 'MINERAL', 'GUNSMOK', 'ARMORY',
       'CLEAR', 'MATTE A', 'MATTE T', 'MATTE S', 'MATTE M', 'MATTE G',
       'SATIN N', 'SATIN W', 'BRUSHED', 'BRUSH', 'SATIN', 'OIL G', '',
       'Matte B', 'Matte D', 'SATIN G', 'MATTE V', 'Black/C', 'Footbal',
       'Dark Gr', 'Soft Pi', 'Midnigh', 'Clear/C', 'GREEN G', 'INDIGO',
       'PLUM GR', 'STADIUM', 'DENIM G', 'OXBLOOD', 'LIGHT S', 'BURNT S',
    

### Prompt (for ChatGPT / LLM)

**Aim:** Get a mapping from raw color strings in our data to standard color categories for modeling.

**Model:** ChatGPT 5.2

**How to use:** 
Run the cell above to get `demand_monthly["Color"].unique()`, then paste that list into the prompt below in place of `<color list>`.

---

**Prompt (copy & replace <color list> with your list):**

> You are a data scientist working on sales prediction for eyewear. We have a "Color" feature with vendor-specific names (often truncated, e.g. BUTTERSC, DARK HAV). Map each of the following raw color labels into **one** standard color category: black, blue, brown, grey, green, gold, silver, red, pink, purple, orange, yellow, beige, white, mint, burgundy, or other, and into **one** standard finish category: shiny(default), matte, satin, brushed. Use lowercase. Avoid ambiguous or creative names (e.g. "wolf", "midnight haze"). For transparent/clear use "crystal"; for patterns (e.g. striped) or truly unclear use "other".
>
> Return **only** a valid Python dictionary: keys = exact raw strings below, values = standard category strings. No explanation.
>
> Raw color list:
> ```
> <color list>
> ```


In [10]:
def extract_color_features(raw_val):
    if pd.isna(raw_val) or str(raw_val).strip() == '':
        return pd.Series([None, None, 0])
    
    full_name = str(raw_val).upper().replace('-', '/').strip()


    finish = 'shiny'
    if any(x in full_name for x in ['MATTE', 'MATT', 'WOLF']):
        finish = 'matte'
    elif 'SATIN' in full_name:
        finish = 'satin'
    elif 'BRUSH' in full_name or 'BRUSHED' in full_name:
        finish = 'brushed'

    parts = [p.strip() for p in full_name.split('/')]
    primary_part = parts[0]
    is_multicolor = 1 if len(parts) > 1 and parts[1] != "" else 0

    def map_to_base(text):
        if not text:
            return "Don't know"
        if any(x in text for x in ['TORTO', 'HAVANA', 'HAVA', 'HAV', 'TOR', 'TOKYO', 'HONEY TO', 'DARK TO', 'SOFT TO', 'BLONDE HA','MATTE T']):
            return 'tortoise'

        if any(x in text for x in ['CRYST', 'CLEAR', 'TRANSPARENT', 'TRANSPAREN', 'TRANSPARE', 'MT CRYS', 'SHINY CRY','MATTE CRYS','CRY']):
            return 'crystal'


        if any(x in text for x in ['BLACK', 'BLAC', 'MATTE BL', 'MATTE B', 'MATT BLAC', 'MATTE BLA', 'OBSIDIA', 'ONYX', 'GRIDIRO','MATTE O']):
            if 'BLUE' not in text:
                return 'black'

        if any(x in text for x in ['GREY', 'GRAY', 'ANTHR', 'ANTHRAC', 'GUNMETAL', 'GUNSMOK', 'SMOKE', 'WOLF', 'OYSTER', 'FOOTBAL', 'MINERAL', 'ARMORY', 'LUMI', 'DARK GR', 'MATTE A', 'MATTE G', 'MATTE S', 'SATIN G','DARK','MATTE D','CHARCOA','STRIPED G']):
            return 'grey'

        if any(x in text for x in ['BLUE', 'NAVY', 'NAV', 'AVIO', 'MIDNIGH', 'AZURE', 'PETROL', 'INDIGO', 'ASTRONOMY', 'DENIM', 'AQUA', 'SATIN B', 'SATIN N', 'MEDITER', 'TURQUOISE', 'MATTE N', 'MATTE M', 'MATTE I', 'MATTE BLU', 'MYSTIC']):
            return 'blue'

        if any(x in text for x in ['BROWN', 'TAUPE', 'MINK', 'CACAO', 'MATTE BUR', 'MATTE BROW', 'LIGHT BRO', 'STRIPED B']):
            return 'brown'

        if any(x in text for x in ['GREEN', 'STADIUM', 'MINT', 'SAGE', 'OLIVE', 'STAD', 'MATTE GREE', 'LIGHT G', 'FOREST', 'VINTAGE']):
            return 'green'

        if any(x in text for x in ['GOLD', 'AMBER', 'ANTIQUE', 'OIL G', 'SATIN A', 'MATTE GO', 'MATTE GOL', 'LIGHT GOL']):
            return 'gold'

        if any(x in text for x in ['RED', 'BURGUNDY', 'BURGUND', 'CHERRY', 'OXBLOOD', 'BRICK', 'UNIVERSITY']):
            return 'red'

        if any(x in text for x in ['PINK', 'ROSE', 'BLUSH', 'MAGENTA', 'FOAM', 'SOFT PI']):
            return 'pink'

        if any(x in text for x in ['BEIGE', 'SAND', 'KHAK', 'KHAKI', 'BIO BEI', 'NUD', 'MILKY', 'MATTE KHAK']):
            return 'beige'

        if any(x in text for x in ['YELLOW', 'BUTTERSC', 'BLONDE', 'CYBER']):
            return 'yellow'

        if any(x in text for x in ['PURPLE', 'VIOLET', 'LILAC', 'PLUM', 'MATTE V', 'MATTE PURPL']):
            return 'purple'

        if any(x in text for x in ['ORANGE', 'PEACH', 'TOTAL', 'BURNT']):
            return 'orange'

        if any(x in text for x in ['SILVER', 'PLATINU','LIGHT S']):
            return 'silver'

        if any(x in text for x in ['WHITE', 'CHALK', 'MATTE W', 'SATIN W']):
            return 'white'

        return 'other'

    color_base = map_to_base(primary_part)
    
    return pd.Series([color_base, finish, is_multicolor])

demand_monthly[['Color_Base', 'Color_Finish', 'Is_Multicolor']] = demand_monthly['Color'].apply(extract_color_features)

# demand_monthly = demand_monthly.dropna(subset=['Color_Base'])

In [11]:
other_or_nan_mask = (demand_monthly['Color_Base'] == 'other') | (demand_monthly['Color_Base'].isna())
other_colors = demand_monthly[other_or_nan_mask]['Color'].unique()

print(f"Found {other_or_nan_mask.sum()} rows with Color_Base = 'other' or NaN")
print("\nUnique Color values for these rows:")
print(other_colors)

Found 3444 rows with Color_Base = 'other' or NaN

Unique Color values for these rows:
['STRIPED' 'MATTE' 'BRUSHED' 'BRUSH' 'SATIN' '' 'SOFT' 'SHIN' 'MATT']


In [12]:
demand_monthly.Size.unique()

array(['57', '55', '56', '52', '54', '51', '48', '59', '53', '49', '58',
       '50', '47', '60', '', '46', '5'], dtype=object)

In [13]:
target_cols = ['Color_Base', 'Color_Finish', 'Size', 'Is_Multicolor']
for col in target_cols:
    demand_monthly[col] = demand_monthly[col].fillna('Unknown')

demand_monthly['Size'] = demand_monthly['Size'].replace(['', '5'], 'Unknown')
demand_monthly.loc[demand_monthly['Color_Base'] == 'other', 'Color_Base'] = 'Unknown'

for col in target_cols:
    unknown_count = (demand_monthly[col] == 'Unknown').sum()
    print(f"{col}: {unknown_count} labelled as Unknown")

# check nan value
print(demand_monthly[target_cols].isna().sum())

Color_Base: 3444 labelled as Unknown
Color_Finish: 3072 labelled as Unknown
Size: 2928 labelled as Unknown
Is_Multicolor: 0 labelled as Unknown
Color_Base       0
Color_Finish     0
Size             0
Is_Multicolor    0
dtype: int64


In [14]:
demand_monthly.loc[
    demand_monthly["Style"].str.contains("OPTICAL"),
    "OpticalOrOptical"
] = "Optical"

demand_monthly.loc[
    ~ demand_monthly["Style"].str.contains("OPTICAL"),
    "OpticalOrOptical"
] = "Sun"

In [15]:
demand_monthly.loc[
    demand_monthly["BrandLine"].str.contains("NIKE"),"BrandName"
] = "Nike"
demand_monthly.loc[
    demand_monthly["BrandLine"].str.contains("LACOSTE"),"BrandName"
] = "Lacoste"
demand_monthly.loc[
    demand_monthly["BrandLine"].str.contains("CALVIN KLEIN"),"BrandName"
] = "Calvin Klein"

In [16]:
demand_monthly.head(25)

,Collection,BrandLine,Material,StyleCode,GridValue,Style,Region,Size,Color,Month,Demand,Color_Base,Color_Finish,Is_Multicolor,OpticalOrOptical,BrandName
0,KC,CALVIN KLEIN SUN,45073,CK20541S,5719001,CK20541S/57/BLACK,AMER,57,BLACK,2023-09,26.0,black,shiny,0.0,Sun,Calvin Klein
1,KC,CALVIN KLEIN SUN,45073,CK20541S,5719001,CK20541S/57/BLACK,AMER,57,BLACK,2023-10,28.0,black,shiny,0.0,Sun,Calvin Klein
2,KC,CALVIN KLEIN SUN,45073,CK20541S,5719001,CK20541S/57/BLACK,AMER,57,BLACK,2023-11,27.0,black,shiny,0.0,Sun,Calvin Klein
3,KC,CALVIN KLEIN SUN,45073,CK20541S,5719001,CK20541S/57/BLACK,AMER,57,BLACK,2023-12,26.0,black,shiny,0.0,Sun,Calvin Klein
4,KC,CALVIN KLEIN SUN,45073,CK20541S,5719001,CK20541S/57/BLACK,AMER,57,BLACK,2024-01,9.0,black,shiny,0.0,Sun,Calvin Klein
5,KC,CALVIN KLEIN SUN,45073,CK20541S,5719001,CK20541S/57/BLACK,AMER,57,BLACK,2024-02,24.0,black,shiny,0.0,Sun,Calvin Klein
6,KC,CALVIN KLEIN SUN,45073,CK20541S,5719001,CK20541S/57/BLACK,AMER,57,BLACK,2024-03,21.0,black,shiny,0.0,Sun,Calvin Klein
7,KC,CALVIN KLEIN SUN,45073,CK20541S,5719001,CK20541S/57/BLACK,AMER,57,BLACK,2024-04,35.0,black,shiny,0.0,Sun,Calvin Klein
8,KC,CALVIN KLEIN SUN,45073,CK20541S,5719001,CK20541S/57/BLACK,AMER,57,BLACK,2024-05,21.0,black,shiny,0.0,Sun,Calvin Klein
9,KC,CALVIN KLEIN SUN,45073,CK20541S,5719001,CK20541S/57/BLACK,AMER,57,BLACK,2024-06,20.0,black,shiny,0.0,Sun,Calvin Klein


In [20]:
demand_monthly.isna().sum()

Collection          0
BrandLine           0
Material            0
StyleCode           0
GridValue           0
Style               0
Region              0
Size                0
Color               0
Month               0
Demand              0
Color_Base          0
Color_Finish        0
Is_Multicolor       0
OpticalOrOptical    0
BrandName           0
dtype: int64

## 2.Merge the products sheet

In [17]:
# load and merge all the products sheet into one dataframe, only keep the common columns

def _norm_cols(df):
    df.columns = [str(c).strip().upper() for c in df.columns]
    return df

product_sheets = [
    ("Calvin Klein", "Calvin Klein_Sept24 ATP.xlsx", "SUN"),
    ("Calvin Klein", "Calvin Klein_Sept24 ATP.xlsx", "OPH"),
    ("Lacoste", "LACOSTE_Sept24 ATP.xlsx", "LACOSTE OPTICAL"),
    ("Lacoste", "LACOSTE_Sept24 ATP.xlsx", "LACOSTE SUN"),
    ("Nike", "Nike_Sept24 ATP.xlsm", "Nike Sept 24 Optical"),
    ("Nike", "Nike_Sept24 ATP.xlsm", "Nike Sept 24 Sun"),
]

dfs = []
for brand, fname, sheet in product_sheets:
    df = pd.read_excel(base / fname, sheet_name=sheet)
    df = _norm_cols(df)
    
    df["_BRAND"] = brand
    df["_SHEET_TYPE"] = "Optical" if any(x in sheet.upper() for x in ["OPTICAL", "OPH"]) else "Sun"
    
    dfs.append(df)

common_cols = set(dfs[0].columns)
for df in dfs[1:]:
    common_cols = common_cols.intersection(set(df.columns))

parts = [df[list(common_cols)] for df in dfs]
products = pd.concat(parts, ignore_index=True)

print("shared cols:", list(products.columns))
print("shape", products.shape)

shared cols: ['MATERIALNUMBER', 'GENDER', 'COLORDESCRIPTION', 'FAMILY', 'BRAND', 'MATERIALCODE2', '_SHEET_TYPE', 'RECOMMENDEDREASONS', 'PROTOTYPECODE', 'RXABLE', 'EUROWHOLESALEPRICE', 'USRETAILPRICE', 'MADEIN', 'RELEASEDATE', 'IMAGE', 'FIT', 'MATERIALCODE1', 'USWHOLESALEPRICE', 'EURORETAILPRICE', 'FRAMECONSTRUCTION', 'BASECURVE', 'NOTES', '_BRAND', 'COLORCODE', 'SUNOPTICAL', 'FRAMESHAPE', 'SIZES', 'COLORADD']
shape (316, 28)


In [18]:
print(products.head())

  MATERIALNUMBER GENDER COLORDESCRIPTION      FAMILY         BRAND  \
0       CK24110S      F            BLACK  AVANTGARDE  CALVIN KLEIN   
1       CK24110S      F   LIGHT GUNMETAL  AVANTGARDE  CALVIN KLEIN   
2       CK24110S      F            BROWN  AVANTGARDE  CALVIN KLEIN   
3       CK24110S      F       LIGHT GOLD  AVANTGARDE  CALVIN KLEIN   
4       CK24111S      U      MATTE BLACK  AVANTGARDE  CALVIN KLEIN   

  MATERIALCODE2 _SHEET_TYPE RECOMMENDEDREASONS PROTOTYPECODE  RXABLE  ...  \
0         METAL         Sun                ADV      E11397D1    True  ...   
1         METAL         Sun      Potential ADV      E11397D1    True  ...   
2         METAL         Sun      Potential ADV      E11397D1    True  ...   
3         METAL         Sun      Potential ADV      E11397D1    True  ...   
4         METAL         Sun                NaN      E11397A1    True  ...   

  EURORETAILPRICE FRAMECONSTRUCTION BASECURVE NOTES        _BRAND COLORCODE  \
0          190,00          FULL RIM  

In [19]:
# diagnose NaNs and drop columns that are entirely NaN
nan_counts = products.isna().sum()
print("NaN counts in products:")
print(nan_counts)

all_nan_cols = nan_counts[nan_counts == len(products)].index.tolist()
print(f"dropping {len(all_nan_cols)} all-NaN columns: {all_nan_cols}")
products = products.drop(columns=all_nan_cols)
print("new shape:", products.shape)
print("remaining cols:", list(products.columns))

NaN counts in products:
MATERIALNUMBER          0
GENDER                  0
COLORDESCRIPTION        0
FAMILY                  0
BRAND                   0
MATERIALCODE2           0
_SHEET_TYPE             0
RECOMMENDEDREASONS    181
PROTOTYPECODE           0
RXABLE                  0
EUROWHOLESALEPRICE      0
USRETAILPRICE           0
MADEIN                  0
RELEASEDATE             0
IMAGE                 316
FIT                     0
MATERIALCODE1           0
USWHOLESALEPRICE        0
EURORETAILPRICE         0
FRAMECONSTRUCTION       0
BASECURVE               0
NOTES                 302
_BRAND                  0
COLORCODE               0
SUNOPTICAL              0
FRAMESHAPE              0
SIZES                   0
COLORADD              316
dtype: int64
dropping 2 all-NaN columns: ['IMAGE', 'COLORADD']
new shape: (316, 26)
remaining cols: ['MATERIALNUMBER', 'GENDER', 'COLORDESCRIPTION', 'FAMILY', 'BRAND', 'MATERIALCODE2', '_SHEET_TYPE', 'RECOMMENDEDREASONS', 'PROTOTYPECODE', 'RXABLE',